# 📘基于 DeepSeek 打造 Agent+RAG 问答系统
感谢 [SeasonMay](https://github.com/opensourcedot/xihe-cases/pull/37) 对本文的贡献。

## 背景介绍
DeepSeek（杭州深度求索人工智能基础技术研究有限公司）成立于 2023 年 7 月 17 日，是一家专注于大语言模型（LLM）及相关技术研发的创新型科技公司。公司由知名私募机构幻方量化孵化，凭借数据蒸馏技术，提炼出更高质量、更具价值的数据，以提升模型的训练效率和性能。
## 技术演进
自 2024 年以来，DeepSeek 持续推进大模型的迭代升级，先后发布了以下关键模型：

* 2024 年 1 月至 6 月：推出 DeepSeek-LLM、DeepSeek-Coder、DeepSeekMath、DeepSeek-VL、DeepSeek-V2、DeepSeek-Coder-V2；
* 2024 年 9 月 5 日：整合 DeepSeek-Coder-V2 与 DeepSeek-V2-Chat，发布 DeepSeek-V2.5；
* 2024 年 12 月 13 日：发布多模态模型 DeepSeek-VL2；
* 2024 年 12 月 26 日：正式开源旗舰模型 DeepSeek-V3。

这些成果体现了 DeepSeek 在通用语言建模、代码生成、多模态理解等方向的强大研发能力。

## 项目介绍
本项目基于 DeepSeek 先进的大语言模型，构建一个 **Agent + RAG（Retrieval-Augmented Generation）问答系统**，可智能解析并回答来自《MindSpore 设计概览》PDF 文档中的内容。

该系统不仅展示了大语言模型在专业领域文档理解和信息提取方面的能力，同时也为开发者提供了一个实践范例，说明如何结合预训练模型与特定领域知识，打造高效的问答解决方案。

### ✅ Step 1：安装依赖包
本项目主要基于 MindSpore NLP 和 Mindspore 进行开发，构建面向专业文档的问答系统。

**MindSpore NLP** 是一个基于 MindSpore 深度学习框架的开源自然语言处理（NLP）库，旨在为各种 NLP 任务提供高效、灵活的解决方案。该库集成了众多常用的 NLP 模型和方法，使研究人员和开发者能够更便捷地构建、训练和部署 NLP 应用，加速技术创新和落地。

在开始构建项目之前，请先安装以下依赖项：

In [1]:
%%capture captured_output
# 实验环境已经预装了mindspore==2.6.0，如需更换mindspore版本，可更改下面 MINDSPORE_VERSION 变量
!pip uninstall mindspore -y
%env MINDSPORE_VERSION=2.6.0
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/${MINDSPORE_VERSION}/MindSpore/unified/aarch64/mindspore-${MINDSPORE_VERSION}-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

In [2]:
# 查看当前 mindspore 版本
!pip show mindspore

Name: mindspore
Version: 2.6.0
Summary: MindSpore is a new open source deep learning training/inference framework that could be used for mobile, edge and cloud scenarios.
Home-page: https://www.mindspore.cn
Author: The MindSpore Authors
Author-email: contact@mindspore.cn
License: Apache 2.0
Location: /home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages
Requires: asttokens, astunparse, dill, numpy, packaging, pillow, protobuf, psutil, safetensors, scipy
Required-by: mindnlp


In [3]:
%%capture captured_output
!pip install pymupdf sentence-transformers faiss-cpu mindnlp==0.4.1 newspaper3k lxml[html_clean] jieba==0.39

### ✅ Step 2：RAG——加载 PDF 文档

#### RAG介绍
大语言模型（LLMs） 在智能交互、语言理解等任务中展现出强大的能力，但在实际业务场景中仍面临以下挑战：

1. **知识更新滞后**

    大模型的知识来源受限于其训练数据，当前主流大模型（如 ChatGPT、文心一言、通义千问等）主要基于网络公开数据进行训练，导致难以获取实时、私有或离线数据，意味着它们往往难以获取实时、非公开或离线数据中的知识，从而限制了模型在某些专业领域的应用。

2. **幻觉问题（Hallucination）**

    由于 LLM 本质上基于概率建模，输出实质上是一系列数值运算的结果。在某些情况下，这可能导致模型在不擅长的场景或缺乏相关知识时产生误导性的回答。这种幻觉问题的识别需要使用者具备相应领域的知识，从而限制了使用的效果。

3. **数据安全与隐私**

    数据安全性也是现代社会关注的焦点，企业或个人通常不愿上传私有数据到第三方平台，担心数据泄露风险。这也进一步限制了大模型在企业级业务中的落地能力。

为了解决上述问题，检索增强生成（Retrieval Augmented Generation ，RAG）技术应运而生。它将信息检索与自然语言生成相结合，让大模型不仅依赖其“记忆”，还能借助外部知识库来回答问题。

RAG 系统核心结构可概括为两部分：
> 向量检索（Retrieval） + 大模型生成（Generation）

在向量检索部分，系统使用向量数据库对文档进行编码与检索；而在生成部分，借助大模型对补充背景信息的用户输入进行语义理解与答案生成。
![图片1](https://matuimg.com/i/2025/03/31/10ovsfn.png)

**RAG 工作流程如下**：
1. 检索阶段

    用户提出问题后，系统首先从向量数据库中检索出与查询高度相关的文档片段。

2. 生成阶段

    检索结果与用户原始输入被拼接后作为上下文输入大模型，生成包含真实信息的高质量回答。

这一机制可显著降低幻觉风险，提升回答的准确性和可控性。

在本案例中，我们基于《MindSpore 设计概览》PDF 文档构建知识库，通过 RAG 技术实现对该文档内容的智能问答。

你可以使用以下代码下载并加载该 PDF 文档以构建本地语义索引，用于后续的向量检索和问答生成任务：

In [4]:
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/white_paper/MindSpore_white_paperV1.1.pdf"

# 下载 《MindSpore 设计概览》 PDF 文档
download(url, "MindSpore_Design_Overview.pdf", replace=True)


file_sizes: 100%|███████████████████████████| 1.83M/1.83M [00:00<00:00, 192MB/s]
Successfully downloaded file to MindSpore_Design_Overview.pdf


'MindSpore_Design_Overview.pdf'

In [5]:
import fitz  # PyMuPDF
pdf_path = 'MindSpore_Design_Overview.pdf'

doc = fitz.open(pdf_path)
pdf_text = [page.get_text() for page in doc]

### ✅ Step 3：清洗 PDF 文本

In [6]:
import pandas as pd

# 定义文本清洗函数：将文本按空格切分并重新连接，去除多余空白字符
def clean_text(text):
    return ' '.join(text.split())

# 构建一个 DataFrame，用于存储每页 PDF 的页码与对应的清洗后文本
pdf_df = pd.DataFrame({
    'page': list(range(1, len(pdf_text)+1)),
    'text': [clean_text(t) for t in pdf_text]
})

### ✅ Step 4：切分为语义块

In [7]:
chunks = []
# 遍历每一页的文本内容
for _, row in pdf_df.iterrows():
    text = row['text']
    # 按固定长度（300 个字符）对文本进行滑动窗口式切分
    for i in range(0, len(text), 300):
        chunk = text[i:i+300]
        chunks.append(chunk)

### ✅ Step 5：向量化并构建 FAISS 检索器

在实际应用中，文档往往包含成千上万的字符，对于大语言模型而言，直接处理整篇长文本可能面临上下文长度限制，进而影响理解效果，尤其在提取关键信息时可能出现遗漏或误判。为应对这一挑战，常用的做法是采用**文本切分策略**，将大型文档拆分为多个较小的片段（chunk）。

本项目中，我们也采用了这一方法：将原始文档划分为多个小段，每段包含固定数量的字符，并在段与段之间引入部分字符的重叠。这样可以在避免上下文过长的同时，确保一个完整语义单元的上下文不会因切分而被截断，从而提升后续问答系统的信息提取准确率和鲁棒性。

完成文本切分后，我们对每个文本块进行向量化处理，即将自然语言内容转换为稠密向量（embedding），从而使其可以参与后续的相似度检索。向量化模型众多，包括：
* Openai的ChatGPT-Embedding
* 百度的ERNIE-Embedding V1
* 北京智源研究院推出的 BGE 模型
* 以及本项目采用的轻量级模型 paraphrase-multilingual-MiniLM-L12-v2

文本向量生成后，我们会将所有向量存入一个**向量数据库**中，并构建索引结构，以便在系统运行时根据用户问题快速检索出最相关的文本片段，实现“以文搜文”的功能。这一过程，也被称作**数据入库**。在 RAG（Retrieval-Augmented Generation，检索增强生成）系统中，常用的向量数据库包括：
* FAISS：轻量、适用于本地部署的小型向量库
* ChromaDB：支持嵌入文档管理的轻量数据库
* Elasticsearch：传统搜索引擎，支持向量检索插件
* Milvus：面向大规模向量数据的分布式向量数据库

在选择适合的数据库时，应全面考虑业务的具体需求、硬件配置以及性能要求等诸多因素，以确保选出最适合的数据库方案。本项目采用较通用常见的**FAISS**进行向量入库与索引构建，以满足本地化部署和快速原型开发的需求。


In [8]:
### 建议下载到本地部署，可以从魔塔或者昇思大模型平台下载
!git lfs install
!git clone https://www.modelscope.cn/Ceceliachenen/paraphrase-multilingual-MiniLM-L12-v2.git

Git LFS initialized.
fatal: 目标路径 'paraphrase-multilingual-MiniLM-L12-v2' 已经存在，并且不是一个空目录。


In [9]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 加载 SentenceTransformer 嵌入模型
embed_model = SentenceTransformer('./paraphrase-multilingual-MiniLM-L12-v2')
# 将每个文本片段（chunk）编码为稠密向量（embedding）
chunk_embeddings = embed_model.encode(chunks)

# 构建一个基于 L2 距离的向量索引器
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
# 将所有文本片段的向量添加到索引中
index.add(np.array(chunk_embeddings))

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### ✅ Step 6：定义检索接口

In [10]:
def query_pdf(query, top_k=3):
    # 将用户输入的问题（query）编码为向量
    q_vec = embed_model.encode([query])
    
    # 在向量索引中查找与查询向量最相近的 top_k 个文档片段
    D, I = index.search(np.array(q_vec), top_k)    # D：距离，I：索引列表
    
    # 根据检索结果的索引，从原始 chunks 中提取对应的文本片段
    return [chunks[i] for i in I[0]]

### ✅ Step 7：加载 Deepseek 模型
本项目采用了 DeepSeek-R1-Distill-Qwen-7B 模型 —— 一种经过蒸馏压缩的小规模大语言模型，具备较高的推理效率和良好的语言理解能力，适用于资源有限场景下的智能问答任务。

In [11]:
import mindspore as ms
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer

# 加载模型
model_id = "MindSpore-Lab/DeepSeek-R1-Distill-Qwen-7B"
tokenizer = AutoTokenizer.from_pretrained(model_id, mirror='modelers', ms_dtype=ms.float16)
model = AutoModelForCausalLM.from_pretrained(model_id, mirror='modelers', ms_dtype=ms.float16)
# 设置为推理模式
model.set_train(False)

def qwen_generate(prompt):
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="ms")
    # 调用模型的 generate 方法，生成新的 tokens
    outputs = model.generate(**inputs, max_new_tokens=256)
    # 去除原始输入部分，只保留模型生成的内容
    output = [o[len(i):] for i, o in zip(inputs.input_ids, outputs)]
    # 解码为可读文本
    return tokenizer.batch_decode(output, skip_special_tokens=True)[0]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.076 seconds.
Prefix dict has been built succesfully.
Qwen2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### ✅ Step 8：集成工具：时间、翻译、网页摘要

在人类文明的发展历程中，工具的发明与使用始终代表着智慧的跃升。工具不仅扩展了人类的能力，也极大地提升了我们解决复杂问题的效率与生产力。随着任务复杂度不断提升，我们更依赖于工具的辅助，以腾出时间与资源，专注于更具价值的目标。

同样地，大语言模型（LLMs）虽然具备丰富的通识知识与优秀的语言理解与生成能力，但在真实场景中仍面临一些局限，例如：
* 预训练数据具有滞后性，无法感知最新动态信息；
* 私域数据受限于安全与隐私要求，无法直接访问；
* 存在幻觉（hallucination），生成内容缺乏依据或可信度。

即便引入了 RAG（检索增强生成）机制，在一些场景中依然难以满足复杂任务的实时性和准确性需求。因此，正如人类依赖工具拓展自身能力，大模型也需要具备**调用外部工具**的能力，进一步增强其实用性与交互性。引入外部工具调用机制，可以带来以下几方面的增强：
* 接入动态数据源（如当前时间、网页内容），提升输出的实时性与上下文关联性；
* 打破单一文本生成范式，实现跨模态、多任务集成（如结构化摘要、翻译、表格处理等）；
* 增强模型交互性与任务执行力，从单一问答助手进化为可执行 agent。

通过工具的集成，大模型不再只是一个被动响应的**语言系统**，而真正成为一个具备主动任务能力的**智能体（Agent）**。

为了展示 LLM 与工具协作的基本能力，本项目集成了以下几类通用工具，供智能体按需调用：

* **时间查询工具**：用于提供当前系统时间，解决时效性问题
* **网页摘要工具**：抓取指定网页内容并生成摘要
* **中英互译工具**：用于对话或内容的实时翻译

In [12]:
from newspaper import Article
from datetime import datetime, timezone, timedelta


def get_time_info(query):
    # 设置为 UTC+8 时区的时间
    tz_utc_8 = timezone(timedelta(hours=8))
    now = datetime.now(tz_utc_8)  # 获取带时区信息的当前时间

    if "几点" in query:
        return f"现在是 {now.strftime('%Y-%m-%d %H:%M:%S')}"
    elif "星期" in query:
        weekdays = ["一", "二", "三", "四", "五", "六", "日"]
        weekday_cn = weekdays[now.isoweekday() - 1]  # isoweekday(): 1~7（周一到周日）
        return f"今天是星期{weekday_cn}，{now.strftime('%Y-%m-%d')}"
    elif "几号" in query or "日期" in query:
        return f"今天是 {now.strftime('%Y-%m-%d')}"
    
    # 默认提示信息
    return "我可以告诉你时间哦~"


def translate(query):
    if "翻译" in query:
        # 确定翻译方向（中译英/英译中）
        target = "英文" if "英文" in query else "中文"
        # 清理查询中的翻译指令关键词
        content = query.replace("翻译", "").replace("成英文", "").replace("成中文", "")
        # 调用模型进行翻译
        return qwen_generate(f"请将以下内容翻译成{target}：{content}")
    return "翻译请求格式不清晰。"


def summarize_web(url):
    try:
        # 创建文章对象并设置语言为中文
        article = Article(url, language='zh')
        # 下载并解析网页内容
        article.download()
        article.parse()
        # 调用Qwen模型进行摘要生成
        return qwen_generate("请总结以下网页内容：" + article.text[:2000])
    except Exception as e:
        return f"❌ 网页摘要失败：{e}"

### ✅ Step 9：Agent 路由控制器

In [13]:
class PDFAgent:
    def answer(self, query):
        """
        根据用户输入内容智能分发处理任务
        处理逻辑分支:
        1. 数学计算 → 调用Qwen模型求解
        2. 时间查询 → 调用时间信息模块
        3. 翻译请求 → 调用翻译模块
        4. 网页链接 → 生成网页摘要
        5. 其他问题 → PDF文档检索+Qwen生成答案
        """
        if any(k in query for k in ["计算", "+", "-", "*", "/"]):
            return qwen_generate(f"请计算：{query}")
        elif any(k in query for k in ["几点", "星期", "几号"]):
            return get_time_info(query)
        elif "翻译" in query:
            return translate(query)
        elif query.startswith("http"):
            return summarize_web(query)
        else:
            # 1. 从PDF库检索相关内容
            retrieved = "\n".join(query_pdf(query))
            # 2. 结合上下文生成回答
            return qwen_generate(f"以下是相关资料：{retrieved}\n请回答：{query}")

agent = PDFAgent()

### ✅ Step 10：测试示例

#### RAG知识库背景下的问答示例

In [14]:
print(agent.answer("什么是MindSpore？"))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


嗯，我现在要回答关于MindSpore的问题。首先，我得仔细阅读提供的资料，了解MindSpore的基本信息和特点。资料中提到，MindSpore是一种全场景覆盖的深度学习计算框架，目标是易开发、高效执行和全场景覆盖。

资料里还详细介绍了MindSpore的自动微分机制，使用了源码转换（SCT）和自动微分（AD）技术。这使得构建复杂的模型变得容易，因为可以自动处理流程控制。此外，MindSpore还支持静态编译优化，提升性能。

框架由几个主要组件组成：MindExpression（ME）、MindCompiler（MC）、MindData（MD）、MindRE 和MindArmour（MA）。技术贡献包括自动微分、静态编译优化、计算图构建、软硬件协同优化和多场景支持。

现在，我需要将这些信息整合成一个简明扼要的解释。首先，概述MindSpore的全场景覆盖和其核心优势，然后详细说明其自动微分机制，接着介绍框架的组件和主要技术贡献。确保用简单易懂的语言，避免专业术语过多，同时涵盖所有关键点。

可能的结构是：
1. 引言：


#### 时间工具调用的示例

In [15]:
print(agent.answer("现在几点？"))

现在是 2025-06-05 18:06:15


In [16]:
print(agent.answer("今天是星期几？"))

今天是星期四，2025-06-05


#### 中英互译工具调用的示例

In [17]:
print(agent.answer("翻译成英文：你好，我是AI助手"))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


好，用户让我翻译“你好，我是AI助手”成英文。首先，我得确认这句话的结构和意思。中文里“你好”通常翻译成“Hello”，这是比较常见的问候方式。“我是AI助手”这部分需要准确传达出我是人工智能助手的身份，同时保持礼貌和自然。

“我是AI助手”中的“我是”可以翻译为“I am”或者“I am your AI assistant”，这样更直接明了。“AI助手”直译是“AI assistant”，但为了更口语化一点，可能可以考虑用“AI helper”或者“AI assistant”，看哪个更合适。

接下来，考虑整个句子的流畅性。直接翻译的话是“Hello, I am your AI assistant.”，这样既保留了问候的礼貌，也准确传达了身份。不过，为了更自然，可能可以加上“I am here to help you”来补充说明，让整个翻译更完整。

所以，综合起来，翻译成“I am here to help you, my name is AI assistant.”或者“Hello! I'm AI Assistant, how can I help you today?”都可以。前者更直接，后者则更有互动性。

用户的需求是翻译这句话，可能用于问候或者自我介绍。考虑到不同的应用场景，提供
